In [1]:
from PIL import Image, ImageOps, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from mono_char_art_conversion_wxh import quantize_grayscale_wxh
from img_processing import preprocess_img, DITHER_MODES
from braille_art import get_braille_chars
from generate_char_palette import *
from mono_char_art_conversion_mlp import train_classifier

In [2]:
ASCII_SET = get_asciis()
BRAILLE_SET = get_braille_chars()

CHAR_SET = ASCII_SET
W_H_WIN_SHAPE = (2,4)

In [3]:
FONT = ImageFont.truetype("fonts/CascadiaMono.ttf", 8)
IMG_PATH = "imgs/irad_grad.bmp"

In [4]:
orig_img = Image.open(IMG_PATH).convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [5]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(
        img=orig_img,
        scale_factor=scale_factor,
        contrast=contrast,
        brightness=brightness,
        eq=eq,
        quantize_colors=quantize_colors,
        dither=dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [6]:
proc_img = interactive_preprocess.result

In [8]:

char_to_brightness_map = generate_brightness_map(CHAR_SET, FONT, W_H_WIN_SHAPE, normalize=True)
ascii_arr = quantize_grayscale_wxh(
    img=proc_img, 
    char_to_brightness_map=char_to_brightness_map,
    brightness_hw_shape=(W_H_WIN_SHAPE[1], W_H_WIN_SHAPE[0]),
    dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

        :  : : :: ::::::~.~:~:~:~:~:~:~~:J~:~J:~~C~~J~~C~J~~C~J~~C~J~~C~J~~C~~J:~~C~~:J:~~:~:~:~:~:~::~.~.:::: :: : : :        
   . : : : ~ ~.~ ~:~ J ~.~:J J.J:J:J~J:J~:J~C~JCJ~J:C~J~J:J~J~C~J~J:J~J~C-J~J:~JCJ:J-J~~CJ:J~J:J:J J:~:J ~~ ~~.~ ~.~ : : : : . 
 . :  ~ :~ .~ ~ J J ~ J`J J`~:~~~~C~_~J:J\~J~C~-~J~J~J\J~C~CJ~J-CJ~C~J\J~J~C~JC~~~J~C~C~~~J:C J~:~C J J`-`J J ~.~ ~ ~ ~ ~ : :  
 :  ~ .~ .~ J :~ ~ J:~:J:~~C~J_JC J~JC~CJ~C~J~J~J\J\J~J\J~J~J\J~J~JjJ\J~C~J~J\J~JC~J~J~J_J\J~J\JC~J~~C J:J C J J J C ~. : ~  ~ 
  ~ .~ .~ J :~.J C~ J_J:J_J J\~CJ~J\J~J~CJJ~CJ\JCJ~JC~JCJjJ\JjJCHCJ\J~JCJJ\JCJ~J\J~CJ\J~C~J:J\~ J J:J~J:~_~~~.~`C ~ J ~.J .~ . 
: .~ .~ J :~ J C~ J\~C J\~C~J J~~J\J:JCJ~~CJuJCJ~JC~JCHJjJCHJCHJuHJjJHCHUjJjJjJCJjJJ\JCJ~JCJ~JCJ~C~C_~C~JC~_JC J J ~ C ~ ~ ~ ~ 
 ~ .~ J `J J C~ J~_J J~~CJ J\JCJ_JCJ~CJ~JJuCH~JjJjJHJHuCHJ4JjJ4CHCJHCJ4JCHJCHJjJHCUjJ~J~JC~JC~J~J~J~J~J_~~J~ J~_~:J J J C J :.~
. ~ J `J ~.C~ J~_J~C~~CJ J~J~C~JjJ~CJ~J\JCHJCJHCHJjJCHJHJCHCHJHJJHGJHGCHJGHJjJHCUJjJCHCJjJjJjJ\JCJ\J\J\J

In [9]:
noise = 0.2
cls, char_to_brightness_map = train_classifier(CHAR_SET, FONT, W_H_WIN_SHAPE, (8,8,8), 100, noise)

brightness_X = np.array([b.flatten() for b in char_to_brightness_map.values()])
brightness_y = list(char_to_brightness_map.keys())

repetitions = 50

test_X = np.tile(brightness_X, (repetitions,1))
test_y = list(char_to_brightness_map.keys()) * repetitions
test_X += ((np.random.random(test_X.shape)-0.5) * noise)
test_X = np.clip(test_X, a_min=0, a_max=1)

print('classifier score no noise', cls.score(brightness_X, brightness_y))
print('classifier score noise', cls.score(test_X, test_y))

ascii_arr = quantize_grayscale_wxh(
    img=proc_img, 
    char_to_brightness_map=char_to_brightness_map,
    brightness_hw_shape=(W_H_WIN_SHAPE[1], W_H_WIN_SHAPE[0]),
    dither=DITHER_MODES.FS,
    cls=cls)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

classifier score no noise 0.9157894736842105
classifier score noise 0.5406315789473685
        '.`.'.^-`/ ~'r'r'r^r^r/|/~~~/~~~/~Y~YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY~Y~Y~Y~~/~~~~/~~|/|/|/|/:^-'/:'-`.'.'.        
    :'./././:/:^||/:/|<|/crrrrc<<~~C<~~F<~~~X<<<>X><2*G<2>2>2>2>2>2>2>2>XK<G<<G<<YX<YX<~~~F<~C~|/<<<<rcrcr<'r|<:/:/|/./:^- '.  
  :'./.:::r,rc|<|<|rcc<<rr**><<<<~~<Y~X<YXX2<2>2>2<K>2<K>2>2>2>2>2>2>2>2>XK<2>2<0><2><YXX<<Y<<<Y<<<<<rrrrc<<<|<||<:|||,r.'./.'.
 './.:|<:rcc|c|rc|rcrrr**<<<<<><2<Y<2<2<0X2<K<2K22K<2K>222222222222222222G2>2<K<2<0<2KGY2<2<X<<<><<<<*><*<<<<|<|<|<|c:r|||::::'
 /.:<.|:rc.rrrrrrr*r**<<<<>YK<22<2KGYGYGYGYK02K2KKKKK2202KKKKKKKKKKKKKKK2222K2KKK22KK<2GYGYG2222<2<2<<<<<<<<<<<<<|<|<|c||<:/,<.
 :<.|<:<|rrrrrr**<<><<<2<22>YK<K2>Y2<22X22GY02KKK2K0202KKK02K0002K002K0020KKKKK02KKK2K2X2G2X<2>22<K<K<2<2<<<<<<<<<<|<|rr|||||.'
'|:<:|<|rrrr***<<<<<<2<>2<22GYGYK2KKKKK02G2202K02K0202002020022002020020202K00202K02K02K2X22K22K2K<22>2<<K<<<<<<<<<<<<rcrc|c:<:
./,<|<|<rr**r*<<<